In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.firefox import GeckoDriverManager
import pandas as pd
import re
import csv
import os
import time

In [2]:
def normalizar(s):
    reemplazos = (("á", "a"),("é", "e"),("í", "i"),("ó", "o"),("ú", "u"))
    
    for a, b in reemplazos:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

In [3]:
def extraerProyectos(wait,profeW, nombre, apellidos):
    proyectosProfe = []
    try: 
        wait.until(EC.visibility_of_element_located((By.PARTIAL_LINK_TEXT, 'PROYECTOS'))).click()
        grupoProyectos = wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'grupo-proyectos')))
        
        tipo = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'h2.grupo-proyectos__title')))

        numeroProyecto = 1
        numero = 0
        for proyectos in grupoProyectos:
            tipoTexto = tipo[numero].text
            numero += 1
            
            listaProyectos = proyectos.find_elements(By.CSS_SELECTOR, 'li') 
            
            for proyecto in listaProyectos:
                driver.execute_script("arguments[0].scrollIntoView();", proyecto)
                esResponsable = 'No'
                
                listaResponsables = wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'c-proyecto-card__responsables')))
                nombreCompleto = nombre + ' ' + apellidos
        
                if((nombreCompleto.upper() in listaResponsables[numeroProyecto-1].text) or (normalizar(nombreCompleto.upper()) in listaResponsables[numeroProyecto-1].text)):
                    esResponsable = 'Sí'
                
                #Abrimos el enlace del proyecto en otra pestana
                enlace = proyecto.find_element(By.CSS_SELECTOR, "a.c-proyecto-card__title")
                url = enlace.get_attribute("href")
    
                driver.execute_script(f"window.open('{url}');")
                proyectoW = driver.window_handles[2]
                driver.switch_to.window(proyectoW)

                #Sacamos los datos que necesitamos de cada proyecto: lista de financiadores y dinero
                datos = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'financiador')))
                financiadores = datos.find_elements(By.CSS_SELECTOR, 'div:nth-of-type(1)')
                dinero = datos.find_element(By.CSS_SELECTOR, 'div.importe span')
                categoria = driver.find_element(By.CSS_SELECTOR, 'div.content-header__content h2').text

                proyectosProfe.append({
                    'Proyecto': numeroProyecto,
                    'Tipo': tipoTexto,
                    'Categoria': categoria,
                    'Financiadores': [f.text for f in financiadores],
                    'Importe': dinero.text,
                    'Responsable': esResponsable
                })

                numeroProyecto += 1
            
                driver.close()
                driver.switch_to.window(profeW)
                
        
    except TimeoutException:
        print("No tiene proyectos")

    finally:
        return proyectosProfe    

In [4]:
def extraerTesis(wait):
    
    tesisDirigidas = 0
    
    try:
        wait.until(EC.visibility_of_element_located((By.PARTIAL_LINK_TEXT, 'TESIS'))).click()
        titulos_tesis = wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'investigador-tesis__title')))
        
        for titulo in titulos_tesis:
            if("Tesis dirigidas" in titulo.text):
                tesisDirigidas = int(re.findall(r'\d+', titulo.text)[0])

    except TimeoutException:
        print("No tiene tesis")

    finally:
        return tesisDirigidas

In [5]:
def extraerColaboraciones(wait,driver, nombre, apellidos):

    colaboraciones = []

    try:
        #Las esperas son necesarias por dos razones, la primera es porque tanto para pulsar el boton que descarga el excel
        #como para descargar el excel hay que darle un pequeño tiempo.
        #La segunda razon es pa que no se pete la pagina
        wait.until(EC.visibility_of_element_located((By.PARTIAL_LINK_TEXT, 'COLABORACIÓN'))).click()

        time.sleep(1)

        #Dispara el evento de descarga del excel seleccionando el boton como un objeto DataTable
        driver.execute_script("""
        var table = $('#tablaColaboraciones').DataTable();
        table.button('.buttons-excel').trigger();
        """)

        time.sleep(5)
        
        #Leer excel y sacar colaboradores
        nombreArchivo = nombre.upper() + ' ' + apellidos.upper() + ' - Universidad de Valladolid.xlsx'
        ruta = 'C:/Users/elisa/Downloads/' + nombreArchivo
    
        # Verifica si el nombre del archivo viene o no con tildes
        if not(os.path.exists(ruta)):
            ruta = normalizar(ruta)

        print(ruta)
        time.sleep(1)
        
        colaboraciones = leerExcel(ruta)

        time.sleep(1)
        
        
    except Timeoutexception:
        print("No tiene colaboraciones")

    finally:
        return colaboraciones


In [6]:
def extraerIndicadores(wait):
    wait.until(EC.visibility_of_element_located((By.PARTIAL_LINK_TEXT, 'INDICADORES'))).click()
    time.sleep(1)
    publicacionesTotales = 0
    q1Totales = 0
    #Al parecer no todos los catedraticos tienen publicaciones
    try:
        wait.until(EC.visibility_of_element_located((By.PARTIAL_LINK_TEXT, 'PUBLICACIONES CON IMPACTO'))).click()
        time.sleep(1)
                
        #Necesito extraer todos los indicadores
        listaQ1 = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'g.highcharts-color-0 tspan'))) #- Para acceder a los Q1 solamente

        #Sacar publicaciones totales de cada revista
        publicacionesRevista = wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'section.grupo-indicadores a')))
        
        
        for q1, publicaciones in zip(listaQ1, publicacionesRevista):
           
            publicacionesRevistaNumero = int(re.findall(r'\d+', publicaciones.text)[0])
            publicacionesTotales += publicacionesRevistaNumero
            q1Totales += int(q1.text)
            
            time.sleep(1)
            
        print("Publicaciones totales: " , publicacionesTotales)
        print("Q1 totales: " , q1Totales)
        
    except TimeoutException:
        print("---------------------------------------------> NO TIENE PUBLICACIONES")

    finally:
        
        driver.execute_script("window.history.go(-1)")
        time.sleep(1)
        
        return(publicacionesTotales, q1Totales)

In [7]:
def leerExcel(ruta):
    lista = []
    
    ListaColaboraciones = pd.read_excel(ruta, usecols="A:B", skiprows=2, header=None)
    
    # Iterar sobre cada fila del DataFrame
    for indice, row in ListaColaboraciones.iterrows():
        
        # Acceder a la primera columna (colaborador) y a la segunda columna (numero de colaboraciones)
        numero_colaboraciones = row[ListaColaboraciones.columns[1]]
        
        lista.append({'Colaborador' : indice + 1,
                      'Colaboraciones' : numero_colaboraciones
                     })
    return lista

In [8]:
#Adecuar segun el dato que se quiera guardar
def escribirCSV(nombre, apellidos, departamento, proyectos):
    df = pd.DataFrame({"Nombre": [nombre], "Apellidos": [apellidos], "Departamento": [departamento],"Proyectos": [proyectos]})
    
    #Se mira si se ha creado ya el archivo para solo poner la cabecera una vez
    header = not(os.path.exists('datos_catedraticos_proyectos.csv'))
    df.to_csv("datos_catedraticos_proyectos.csv", mode="a", header=header, index=False ,encoding="utf-8")
    
    print("Guardando en csv")

In [9]:
service = FirefoxService(executable_path=GeckoDriverManager().install())
driver = webdriver.Firefox(service=service)

In [10]:
driver.get("https://portaldelaciencia.uva.es")

### Código para saber como estaban los nombres de los enlaces a

In [12]:
link = driver.find_element(By.LINK_TEXT, "INVESTIGADORES/AS")
link.click()
wait = WebDriverWait(driver, timeout=10)
wait2 = WebDriverWait(driver, timeout=3)

In [13]:
with open('profesores_colaboraciones.csv', 'r', encoding='utf-8', newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    next(spamreader, None)
    departamentosW = None
    for row in spamreader:
        elem = wait.until(EC.visibility_of_element_located((By.PARTIAL_LINK_TEXT, row[0])))
        driver.execute_script("arguments[0].scrollIntoView();", elem)
        elem.click()

        #Muestra departamento
        print(row[0])
        departamentosW = driver.window_handles[0]

        investigadores = int(wait.until(EC.visibility_of_element_located((By.ID, "numResultados"))).text.split(" ")[0])

        if (investigadores > 60):
            for i in range(0, (investigadores // 60)):
                verMasBtn = wait.until(EC.visibility_of_element_located((By.ID, "verMasButton")))
                driver.execute_script("arguments[0].scrollIntoView();", verMasBtn)
                verMasBtn.click()
                time.sleep(2)

        for nombre,apellidos in zip(row[1].split(", "), row[2].split(", ")):
                    
            try:
                apellidosSinTilde = normalizar(apellidos)
                posiblesLinks = wait.until(EC.visibility_of_all_elements_located((By.PARTIAL_LINK_TEXT, apellidosSinTilde.upper())))
                profeLink = None
                
            #Si no encuentro al profesor salta excepcion y se salta al profe    
            except TimeoutException:
               try:
                   posiblesLinks = wait2.until(EC.visibility_of_all_elements_located((By.PARTIAL_LINK_TEXT, apellidos.upper())))
                   #nombre = normalizar(nombre)
                   
               except TimeoutException:
                   print('Saltando a: ' +  apellidos)
                   continue 
            
            #Si hay mas de un profe con los mismos apellidos buscar entre esos por nombre
            if len(posiblesLinks) > 1:
                i = 0
                while(not(nombre.upper() in posiblesLinks[i].text) and not(normalizar(nombre).upper() in posiblesLinks[i].text)):
                    i += 1
                    
                profeLink = posiblesLinks[i]
                
            else:
                profeLink = posiblesLinks[0]

            #Clickar el enlace del profesor correcto
            driver.execute_script("arguments[0].scrollIntoView();", profeLink)

            #Abrir la página del catedrático en otra pestaña
            url = profeLink.get_attribute("href")

            driver.execute_script(f"window.open('{url}');")
            profeW = driver.window_handles[1]
            driver.switch_to.window(profeW)

            print(wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "investigador-header__nombre"))).text + " - " + wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "investigador-header__categoria"))).text)

            #Elegir la que se quiera extraer
            
            #proyectos = extraerProyectos(wait2,profeW,nombre,apellidos)
            #tesisDirigidas = extraerTesis(wait2)
            #colaboraciones = extraerColaboraciones(wait2, driver, nombre, apellidos)
            #publicacionesTotales, q1Totales = extraerIndicadores(wait2)
            #escribirCSV(nombre, apellidos, row[0], publicacionesTotales, q1Totales)
            
            driver.close()
            driver.switch_to.window(departamentosW)
            
            
        driver.execute_script("window.history.go(-1)")

Algebra, Análisis Matemático, Geometría y Topología
FELIX
DELGADO DE LA MATA - CATEDRATICOS DE UNIVERSIDAD
ANA JOSE
REGUERA LOPEZ - CATEDRATICOS DE UNIVERSIDAD
MANUEL
NUÑEZ JIMENEZ - CATEDRATICOS DE UNIVERSIDAD
JAVIER
SANZ GIL - CATEDRATICOS DE UNIVERSIDAD
Anatomía y Radiología
ANGEL LUIS
GATO CASADO - CATEDRATICOS DE UNIVERSIDAD
Arquitectura y Tecnología de Computadores


InvalidSessionIdException: Message: WebDriver session does not exist, or is not active
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:199:5
InvalidSessionIDError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:469:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:559:13
assert.session@chrome://remote/content/shared/webdriver/Assert.sys.mjs:37:4
despatch@chrome://remote/content/marionette/server.sys.mjs:315:19
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


In [ ]:
driver.close()